In [1]:
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")

import numpy as np
import pandas as pd

import plotly.express as px

import matplotlib.pyplot as plt   

import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [2]:
df_data=pd.read_csv('data/raw/dataset.csv')

df_data

,ISIN,BI350,BI493,EG056,EG057,ES018,ES019,ES096,F1754,F1769,...,SR231,SR249,ST024,SR752,SR754,SR756,SR760,SR724,SR721,Year
0,CNE100004ZD8,0.0,NaN,25.644,2.219,NaN,NaN,9.090910,NaN,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
1,US00108J1097,0.0,NaN,120.409,47.224,NaN,NaN,NaN,10.0,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
2,US0079031078,0.0,NaN,385.733,37.010,NaN,NaN,33.333302,30.0,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
3,TW0003438008,0.0,NaN,33.397,7.594,NaN,NaN,42.857101,70.0,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
4,CNE100003MM9,0.0,100,64.797,4.375,NaN,0.023,0.000000,60.0,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,US0395872098,0.0,NaN,1353.663,486.827,NaN,NaN,14.285700,0.0,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
344,US54405Q2093,0.0,NaN,1754.480,80.799,NaN,NaN,NaN,0.0,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
345,US3073597037,0.0,NaN,160.608,0.829,NaN,NaN,NaN,NaN,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023
346,CA35954F1080,0.0,NaN,NaN,597.368,NaN,NaN,NaN,NaN,NaN,...,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,2023


In [3]:
df_data.replace("#N/A Field Not Applicable", np.nan, inplace=True)
df_data.replace("#N/A Authorization", np.nan, inplace=True)


C:\Users\Veronica\AppData\Local\Temp\ipykernel_19432\1358017280.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_data.replace("#N/A Field Not Applicable", np.nan, inplace=True)


In [ ]:
df_data[df_data.isnull().any(axis=1)]

In [ ]:
df_fields=pd.read_csv('data/raw/fields.csv')

df_fields

In [ ]:
def actualizar_valores(df1, df2):
    # Crear un diccionario para facilitar la búsqueda
    columna_a_buscar = df2.iloc[:, 0]  # Primera columna de la segunda tabla
    ultima_columna = df2.iloc[:, -1]  # Última columna de la segunda tabla
    
    # Recorrer las columnas de la primera tabla
    for columna in df1.columns[1:]:  # Excluir la primera columna (ISIN)
        if columna in columna_a_buscar.values:
            # Obtener el índice de la coincidencia en la segunda tabla
            indice = columna_a_buscar[columna_a_buscar == columna].index[0]
            if ultima_columna[indice] in ['SFDR', 'Scope','Scope/SFDR', 'Net zero', 'posible']:
                # Actualizar los valores en la primera tabla a 'No data'
                df1[columna] = df1[columna].where(df1[columna].notna(), 'No data')
            elif ultima_columna[indice] in ['ESG Score', 'E Score', 'S Score', 'G Score', 'Sales']:
                # Actualizar los valores en la primera tabla a 0
                df1[columna] = df1[columna].where(df1[columna].notna(), 0)
    
    return df1
# Actualizar los valores en la primera tabla
df_data = actualizar_valores(df_data, df_fields)


In [ ]:
df_data

In [ ]:
df_data[df_data.isnull().any(axis=1)]

In [ ]:
df_company=pd.read_csv('data/raw/companies.csv')

df_company

In [ ]:
merged_df = pd.merge(df_data, df_company[['ISIN', 'Name', 'Industry', 'Mkt Cap']], on='ISIN', how='left')
merged_df 

In [ ]:
columns_df1 = df_data.columns.tolist()
new_columns_order = columns_df1[:1] +['Name'] +['Mkt Cap']+['Industry'] + columns_df1[1:]
merged_df = merged_df[new_columns_order]
merged_df

In [ ]:
merged_df.to_csv('data/merged_data.csv', index=False)